In [1]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

In [2]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [3]:
reverse_label_mapping = {v:int(k)-1 for k,v in label_mapping.items()}

In [4]:
all_date = ["2024_12_21","2024_12_27","2025_01_03","2025_01_10","2025_02_28"]

for date in all_date:
    # 2. 讀取測試資料，只取同樣的 RSSI 欄位
    file_test = f'timestamp_allignment_Balanced_{date}_rtt_logs.csv'

    selected_columns = ['Label',
                        'AP2_Distance (mm)','AP3_Distance (mm)',
                        'AP2_StdDev (mm)','AP3_StdDev (mm)',
                        'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']

    use_train_columns = [
                        'AP2_Distance (mm)','AP3_Distance (mm)',
                        'AP2_StdDev (mm)','AP3_StdDev (mm)',
                        'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']

    df = pd.read_csv(file_test, usecols=selected_columns)

    # 3. 填補缺失、映射成數值標籤
    df = df.groupby('Label').apply(lambda g: g.fillna(g.mean())).reset_index()

    y_true = df['Label'].map(reverse_label_mapping).values

    # 4. 標準化
    scaler = joblib.load('scaler_test.pkl')   # 對應你存的 scaler 檔名
    X = df[use_train_columns].values
    X_scaled = scaler.transform(X)

    allmde = []
    allaccuracy = []

    for i in range(10):
        # 5. 載入模型並預測
        model = load_model(f'2mcAPbestbset_{i}.h5')   # 換成你要測試的檔案
        y_prob = model.predict(X_scaled)
        y_pred = np.argmax(y_prob, axis=1)

        # 6. 計算 Accuracy
        acc = accuracy_score(y_true, y_pred)
        print(f"Test Accuracy: {acc:.4f}")

        # 7. 計算 MDE
        #    先把 numeric label 轉回字串，再對應座標
        y_pred_labels = [ label_mapping[str(i+1)] for i in y_pred ]
        y_true_labels = [ label_mapping[str(i+1)] for i in y_true ]
        coords_pred = np.array([ label_to_coordinates[l] for l in y_pred_labels ])
        coords_true = np.array([ label_to_coordinates[l] for l in y_true_labels ])
        distances = np.linalg.norm(coords_pred - coords_true, axis=1)
        mde = distances.mean()
        print(f"Test MDE: {mde:.4f} m")

        allmde.append(mde)
        allaccuracy.append(acc)

    print([round(float(mde), 4) for mde in allmde])
    print("平均 MDE:", round(sum(map(float, allmde)) / len(allmde), 4))

    print([round(float(acc), 4) for acc in allaccuracy])
    print("平均 Accuracy:", round(sum(map(float, allaccuracy)) / len(allaccuracy), 4))




c:\Users\吳定洋\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step


Test Accuracy: 0.2939
Test MDE: 1.3209 m
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step
Test Accuracy: 0.3327
Test MDE: 1.2580 m


590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 793us/step
Test Accuracy: 0.3275
Test MDE: 1.3107 m


590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 805us/step
Test Accuracy: 0.3339
Test MDE: 1.2428 m


...

622/622 ━━━━━━━━━━━━━━━━━━━━ 1s 793us/step
Test Accuracy: 0.1526
Test MDE: 1.3460 m


622/622 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step
Test Accuracy: 0.1567


Test MDE: 1.3753 m
622/622 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Test Accuracy: 0.1755
Test MDE: 1.2385 m


622/622 ━━━━━━━━━━━━━━━━━━━━ 1s 797us/step
Test Accuracy: 0.1507


Test MDE: 1.4050 m
622/622 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step
Test Accuracy: 0.1477
Test MDE: 1.3727 m
[1.4479, 1.339, 1.2976, 1.3389, 1.3393, 1.346, 1.3753, 1.2385, 1.405, 1.3727]
平均 MDE: 1.35
[0.1539, 0.178, 0.1608, 0.1623, 0.1644, 0.1526, 0.1567, 0.1755, 0.1507, 0.1477]
平均 Accuracy: 0.1603
